In [3]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
import time
import logging
from bs4 import BeautifulSoup

# Initialize the Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)
visited_urls = set()

save_directory = "retrieved"

def safe_filename(url):
    """Converts the URL into a safe filename by removing the protocol and replacing special characters."""
    return url.replace("http://", "").replace("https://", "").replace("/", "_").replace(":", "_").replace("#", "_")

def fetch_and_save_content(driver, url, depth=0, max_depth=5):
    """Fetches content and saves it, then recursively follows links up to a specified depth."""

    if url in visited_urls or depth > max_depth:
        return

    visited_urls.add(url)

    try:
        print("fetching url: " + url)
        driver.get(url)
        time.sleep(1)  # Wait for JavaScript to run and render the content
        try:
            content = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
        except:
            content = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')


        soup = BeautifulSoup(content, 'html.parser')
        for script in soup(["script", "link", "nav", "footer", "head"]):
            script.extract()  # Remove the tag
        content = str(soup)

        # Save content to file
        filename = save_directory + "/" + safe_filename(url) + ".html"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(content)

        try:
            links = [link.get_attribute('href') for link in driver.find_elements(By.TAG_NAME, 'a')]
        except StaleElementReferenceException:
            # Re-attempt to get the links
            links = [link.get_attribute('href') for link in driver.find_elements(By.TAG_NAME, 'a')]

        for link in links:
            print('links ' + link)
            if link.__contains__(url) and not link.__contains__('#'):  # Ensuring we're only visiting internal links
                fetch_and_save_content(driver, link, depth+1, max_depth)

    except:
        logging.exception("message")
        print(f"Couldn't access {url}")

fetch_and_save_content(driver, "https://plaid.com/docs/", 0, 10)


fetching url: https://plaid.com/docs/
links https://plaid.com/docs/
links https://plaid.com/docs/
links https://plaid.com/docs/quickstart/
fetching url: https://plaid.com/docs/quickstart/
links https://plaid.com/docs/
links https://plaid.com/docs/
links https://plaid.com/docs/quickstart/
links https://plaid.com/docs/quickstart/#introduction
links https://plaid.com/docs/quickstart/#quickstart-setup
links https://plaid.com/docs/quickstart/#setting-up-without-docker
links https://plaid.com/docs/quickstart/#create-your-first-item
links https://plaid.com/docs/quickstart/#how-it-works
links https://plaid.com/docs/quickstart/#making-api-requests
links https://plaid.com/docs/quickstart/#next-steps
links https://plaid.com/docs/quickstart/glossary/
fetching url: https://plaid.com/docs/quickstart/glossary/
links https://plaid.com/docs/
links https://plaid.com/docs/
links https://plaid.com/docs/quickstart/
links https://plaid.com/docs/quickstart/glossary/
links https://plaid.com/docs/quickstart/gl